# 如何使用和开发微信聊天机器人的系列教程
# A workshop to develop & use an intelligent and interactive chat-bot in WeChat

### WeChat is a popular social media app, which has more than 800 million monthly active users.

<img src='http://www.kudosdata.com/wp-content/uploads/2016/11/cropped-KudosLogo1.png' width=30% style="float: right;">
<img src='reference/WeChat_SamGu_QR.png' width=10% style="float: right;">

### http://www.KudosData.com

by: Sam.Gu@KudosData.com


May 2017 ========== Scan the QR code to become trainer's friend in WeChat ========>>

### 第四课：自然语言处理：语义和情感分析
### Lesson 4: Natural Language Processing 2
* 消息文字中名称实体的识别 (Name-Entity detection)
* 消息文字中语句的情感分析 (Sentiment analysis, Sentence level)
* 整篇消息文字的情感分析 (Sentiment analysis, Document level)
* 语句的语法分析 (Syntax / Grammer analysis)

### Flag to indicate the environment to run this program:

In [1]:
# parm_runtime_env_GCP = True
parm_runtime_env_GCP = False

### Using Google Cloud Platform's Machine Learning APIs

From the same API console, choose "Dashboard" on the left-hand menu and "Enable API".

Enable the following APIs for your project (search for them) if they are not already enabled:
<ol>
<li> Google Translate API </li>
<li> Google Cloud Vision API </li>
<li> Google Natural Language API </li>
<li> Google Cloud Speech API </li>
</ol>

Finally, because we are calling the APIs from Python (clients in many other languages are available), let's install the Python package (it's not installed by default on Datalab)

In [2]:
# Copyright 2016 Google Inc.
# Licensed under the Apache License, Version 2.0 (the "License"); 
# import subprocess
# retcode = subprocess.call(['pip', 'install', '-U', 'google-api-python-client'])
# retcode = subprocess.call(['pip', 'install', '-U', 'gTTS'])

# Below is for GCP only: install audio conversion tool
# retcode = subprocess.call(['apt-get', 'update', '-y'])
# retcode = subprocess.call(['apt-get', 'install', 'libav-tools', '-y'])

### 导入需要用到的一些功能程序库：

In [3]:
import io, os, subprocess, sys, re, codecs, time, datetime, requests, itchat
from itchat.content import *
from googleapiclient.discovery import build

### GCP Machine Learning API Key

First, visit <a href="http://console.cloud.google.com/apis">API console</a>, choose "Credentials" on the left-hand menu.  Choose "Create Credentials" and generate an API key for your application. You should probably restrict it by IP address to prevent abuse, but for now, just  leave that field blank and delete the API key after trying out this demo.

Copy-paste your API Key here:

In [4]:
# Here I read in my own API_KEY from a file, which is not shared in Github repository:
with io.open('../../../API_KEY.txt') as fp: 
    for line in fp: APIKEY = line

# You need to un-comment below line and replace 'APIKEY' variable with your own GCP API key:
# APIKEY='AIzaSyCvxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

In [5]:
# Below is for Google Speech synthesis: text to voice API
# from gtts import gTTS

# Below is for  Google Speech recognition: voice to text API
# speech_service = build('speech', 'v1', developerKey=APIKEY)

# Below is for Google Language Tranlation API
# service = build('translate', 'v2', developerKey=APIKEY)

# Below is for Google Natual Language Processing API
# nlp_service = build('language', 'v1', developerKey=APIKEY)
nlp_service = build('language', 'v1beta2', developerKey=APIKEY)

### 多媒体二进制base64码转换 (Define image pre-processing functions)

In [6]:
# Import the base64 encoding library.
import base64
# Pass the image data to an encoding function.
def encode_image(image_file):
    with io.open(image_file, "rb") as image_file:
        image_content = image_file.read()
# Python 2
    if sys.version_info[0] < 3:
        return base64.b64encode(image_content)
# Python 3
    else:
        return base64.b64encode(image_content).decode('utf-8')

# Pass the audio data to an encoding function.
def encode_audio(audio_file):
    with io.open(audio_file, 'rb') as audio_file:
        audio_content = audio_file.read()
# Python 2
    if sys.version_info[0] < 3:
        return base64.b64encode(audio_content)
# Python 3
    else:
        return base64.b64encode(audio_content).decode('utf-8')


### 机器智能API接口控制参数 (Define control parameters for API)

In [50]:
# API control parameter for Image API:
parm_image_maxResults = 10 # max objects or faces to be extracted from image analysis

# API control parameter for Language Translation API:
parm_translation_origin_language = 'zh' # original language in text: to be overwriten by TEXT_DETECTION
parm_translation_target_language = 'zh' # target language for translation: Chinese


# API control parameter for 消息文字转成语音 (Speech synthesis: text to voice)
parm_speech_synthesis_language = 'zh' # speech synthesis API 'text to voice' language
# parm_speech_synthesis_language = 'zh-tw' # speech synthesis API 'text to voice' language
# parm_speech_synthesis_language = 'zh-yue' # speech synthesis API 'text to voice' language

# API control parameter for 语音转换成消息文字 (Speech recognition: voice to text)
# parm_speech_recognition_language = 'en' # speech API 'voice to text' language
parm_speech_recognition_language = 'cmn-Hans-CN' # speech API 'voice to text' language

# API control parameter for 自然语言处理：语义和情感分析
parm_nlp_extractDocumentSentiment = True 
parm_nlp_extractEntities = True 
parm_nlp_extractEntitySentiment = True # only available in v1beta2; Chinese language zh is not supported yet.
parm_nlp_extractSyntax = True 

### 定义一个调用自然语言处理接口的小功能

In [8]:
# Running Speech API
def KudosData_nlp(text, extractDocumentSentiment, extractEntities, extractEntitySentiment, extractSyntax): 
    # Python 2
#     if sys.version_info[0] < 3: 
#         tts = gTTS(text=text2voice.encode('utf-8'), lang=parm_speech_synthesis_language, slow=False)
    # Python 3
#     else:
#         tts = gTTS(text=text2voice, lang=parm_speech_synthesis_language, slow=False)
        
    request = nlp_service.documents().annotateText(body={
                "document":{
                    "type": "PLAIN_TEXT",
                    "content": text
                    },
                "features": {
                    "extractDocumentSentiment": extractDocumentSentiment,
                    "extractEntities": extractEntities,
                    "extractEntitySentiment": extractEntitySentiment, # only available in v1beta2
                    "extractSyntax": extractSyntax,
                    },
                "encodingType":"UTF8"
                })
    responses = request.execute(num_retries=3)        
    print('\nCompeleted: NLP analysis API')
    return responses

### Start of demo

In [9]:
text4nlp = ''

In [51]:
text4nlp = 'As a data science consultant and trainer with Kudos Data, Zhan GU (Sam) engages communities and schools ' \
           'to help organizations making sense of their data using advanced data science , machine learning and ' \
           'cloud computing technologies. Inspire next generation of artificial intelligence lovers and leaders.'

In [23]:
text4nlp = '作为酷豆数据科学的顾问和培训师，Sam Gu （白黑） 结合社群和教育资源，帮助各大公司使用先进的数据科学，'\
           '机器学习和云计算技术来获取数据洞见。激励下一代人工智能爱好者和领导者。'

In [52]:
responses = KudosData_nlp(text4nlp
                            , parm_nlp_extractDocumentSentiment
                            , parm_nlp_extractEntities
                            , parm_nlp_extractEntitySentiment
                            , parm_nlp_extractSyntax)



Compeleted: NLP analysis API


In [53]:
# print(responses)

### * 消息文字中名称实体的识别 (Name-Entity detection)

In [54]:
# print(responses['entities'])

In [55]:
for i in range(len(responses['entities'])): 
#     print(responses['entities'][i])
    print('')
    print(u'[ 实体 {}: {} ]\n[ 类别 : {} | 重要性 : {} ]'.format(
          i+1
        , responses['entities'][i]['name']
        , responses['entities'][i]['type']
        , responses['entities'][i]['salience']
    ))
#     print(responses['entities'][i]['name'])
#     print(responses['entities'][i]['type'])
#     print(responses['entities'][i]['salience'])
    if 'sentiment' in responses['entities'][i]:
        print(u'[ 褒贬度 : {} | 语彩累积 : {} ]'.format(
              responses['entities'][i]['sentiment']['score']
            , responses['entities'][i]['sentiment']['magnitude']
        ))
#     print(responses['entities'][i]['sentiment'])
    if responses['entities'][i]['metadata'] != {}:
        if 'wikipedia_url' in responses['entities'][i]['metadata']:
            print(responses['entities'][i]['metadata']['wikipedia_url'])


[ 实体 1: data science consultant ]
[ 类别 : PERSON | 重要性 : 0.17521429 ]
[ 褒贬度 : 0 | 语彩累积 : 0 ]

[ 实体 2: trainer ]
[ 类别 : PERSON | 重要性 : 0.17521429 ]
[ 褒贬度 : 0 | 语彩累积 : 0 ]

[ 实体 3: communities ]
[ 类别 : PERSON | 重要性 : 0.1560961 ]
[ 褒贬度 : 0.4 | 语彩累积 : 0.8 ]

[ 实体 4: sense ]
[ 类别 : OTHER | 重要性 : 0.06279698 ]
[ 褒贬度 : 0 | 语彩累积 : 0 ]

[ 实体 5: data ]
[ 类别 : OTHER | 重要性 : 0.05968614 ]
[ 褒贬度 : 0.4 | 语彩累积 : 0.4 ]

[ 实体 6: Kudos Data ]
[ 类别 : OTHER | 重要性 : 0.055152062 ]
[ 褒贬度 : 0 | 语彩累积 : 0 ]

[ 实体 7: data science ]
[ 类别 : OTHER | 重要性 : 0.051547784 ]
[ 褒贬度 : 0 | 语彩累积 : 0 ]

[ 实体 8: schools ]
[ 类别 : ORGANIZATION | 重要性 : 0.04895195 ]
[ 褒贬度 : 0 | 语彩累积 : 0 ]

[ 实体 9: organizations ]
[ 类别 : ORGANIZATION | 重要性 : 0.04895195 ]
[ 褒贬度 : 0 | 语彩累积 : 0 ]

[ 实体 10: Zhan GU ]
[ 类别 : PERSON | 重要性 : 0.041413147 ]
[ 褒贬度 : 0 | 语彩累积 : 0 ]

[ 实体 11: cloud computing technologies ]
[ 类别 : OTHER | 重要性 : 0.033586804 ]
[ 褒贬度 : 0 | 语彩累积 : 0 ]

[ 实体 12: Sam ]
[ 类别 : PERSON | 重要性 : 0.028737757 ]
[ 褒贬度 : 0 | 语彩累积 : 0 ]

[ 实体 13

### * 消息文字中语句的情感分析 (Sentiment analysis, Sentence level)

In [32]:
# print(responses['sentences'])

In [34]:
for i in range(len(responses['sentences'])):
    print('')
    print(u'[ 语句 {}: {} ]\n[ 褒贬度 : {} | 语彩累积 : {} ]'.format(
          i+1
        , responses['sentences'][i]['text']['content']
        , responses['sentences'][i]['sentiment']['score']
        , responses['sentences'][i]['sentiment']['magnitude']
    ))



[ 语句 1: 作为酷豆数据科学的顾问和培训师，Sam Gu （白黑） 结合社群和教育资源，帮助各大公司使用先进的数据科学，机器学习和云计算技术来获取数据洞见。 ]
[ 褒贬度 : 0.8 | 语彩累积 : 0.8 ]

[ 语句 2: 激励下一代人工智能爱好者和领导者。 ]
[ 褒贬度 : 0.2 | 语彩累积 : 0.2 ]


https://cloud.google.com/natural-language/docs/basics

* **score 褒贬度** of the sentiment ranges between -1.0 (negative) and 1.0 (positive) and corresponds to the overall emotional leaning of the text.
* **magnitude 语彩累积** indicates the overall strength of emotion (both positive and negative) within the given text, between 0.0 and +inf. Unlike score, magnitude is not normalized; each expression of emotion within the text (both positive and negative) contributes to the text's magnitude (so longer text blocks may have greater magnitudes).


| Sentiment     | Sample Values |
|:-------------:|:-------------:|
| 明显褒义 Clearly Positive	| "score 褒贬度": 0.8, "magnitude 语彩累积": 3.0  |
| 明显贬义 Clearly Negative	| "score 褒贬度": -0.6, "magnitude 语彩累积": 4.0  |
| 中性 Neutral	| "score 褒贬度": 0.1, "magnitude 语彩累积": 0.0  |
| 混合 Mixed	| "score 褒贬度": 0.0, "magnitude 语彩累积": 4.0  |


### * 整篇消息文字的情感分析 (Sentiment analysis, Document level)

In [35]:
# print(responses['documentSentiment'])

In [37]:
print(u'[ 整篇消息 语种 : {} ]\n[ 褒贬度 : {} | 语彩累积: {} ]'.format(
            responses['language']
            , responses['documentSentiment']['score']
            , responses['documentSentiment']['magnitude']
    ))

[ 整篇消息 语种 : zh ]
[ 褒贬度 : 0.5 | 语彩累积: 1.1 ]


### * 语句的语法分析 (Syntax / Grammer analysis)

In [38]:
for i in range(len(responses['tokens'])): 
    print('')
    print(responses['tokens'][i])


{u'text': {u'content': u'\u4f5c\u4e3a', u'beginOffset': 0}, u'dependencyEdge': {u'headTokenIndex': 15, u'label': u'VMOD'}, u'partOfSpeech': {u'case': u'CASE_UNKNOWN', u'reciprocity': u'RECIPROCITY_UNKNOWN', u'mood': u'MOOD_UNKNOWN', u'form': u'FORM_UNKNOWN', u'gender': u'GENDER_UNKNOWN', u'number': u'NUMBER_UNKNOWN', u'person': u'PERSON_UNKNOWN', u'tag': u'VERB', u'tense': u'TENSE_UNKNOWN', u'aspect': u'ASPECT_UNKNOWN', u'proper': u'NOT_PROPER', u'voice': u'VOICE_UNKNOWN'}, u'lemma': u'\u4f5c\u4e3a'}

{u'text': {u'content': u'\u9177', u'beginOffset': 6}, u'dependencyEdge': {u'headTokenIndex': 2, u'label': u'SUFF'}, u'partOfSpeech': {u'case': u'CASE_UNKNOWN', u'reciprocity': u'RECIPROCITY_UNKNOWN', u'mood': u'MOOD_UNKNOWN', u'form': u'FORM_UNKNOWN', u'gender': u'GENDER_UNKNOWN', u'number': u'NUMBER_UNKNOWN', u'person': u'PERSON_UNKNOWN', u'tag': u'NOUN', u'tense': u'TENSE_UNKNOWN', u'aspect': u'ASPECT_UNKNOWN', u'proper': u'NOT_PROPER', u'voice': u'VOICE_UNKNOWN'}, u'lemma': u'\u9177'}

### End of demo

### 定义一个输出NLP分析结果，用于回复微信消息的小功能：

In [41]:
def KudosData_nlp_generate_reply(responses):
    nlp_reply = u'[ NLP 自然语言处理结果 ]'
    
    # * 整篇消息文字的情感分析 (Sentiment analysis, Document level)
    nlp_reply += '\n'
    nlp_reply += '\n' + u'[ 整篇消息 语种 : {} ]\n[ 褒贬度 : {} | 语彩累积: {} ]'.format(
            responses['language']
            , responses['documentSentiment']['score']
            , responses['documentSentiment']['magnitude']
        )

    # * 消息文字中语句的情感分析 (Sentiment analysis, Sentence level)           
    nlp_reply += '\n'
    for i in range(len(responses['sentences'])):
        nlp_reply += '\n' + u'[ 语句 {}: {} ]\n[ 褒贬度 : {} | 语彩累积 : {} ]'.format(
              i+1
            , responses['sentences'][i]['text']['content']
            , responses['sentences'][i]['sentiment']['score']
            , responses['sentences'][i]['sentiment']['magnitude']
        )
                
    # * 消息文字中名称实体的识别 (Name-Entity detection)
    nlp_reply += '\n'
    for i in range(len(responses['entities'])): 
        nlp_reply += '\n' + u'[ 实体 {}: {} ]\n[ 类别 : {} | 重要性 : {} ]'.format(
              i+1
            , responses['entities'][i]['name']
            , responses['entities'][i]['type']
            , responses['entities'][i]['salience']
        )
        if 'sentiment' in responses['entities'][i]:
            nlp_reply += '\n' + u'[ 褒贬度 : {} | 语彩累积 : {} ]'.format(
                  responses['entities'][i]['sentiment']['score']
                , responses['entities'][i]['sentiment']['magnitude']
            )
        if responses['entities'][i]['metadata'] != {}:
            if 'wikipedia_url' in responses['entities'][i]['metadata']:
                nlp_reply += '\n' + responses['entities'][i]['metadata']['wikipedia_url']
                           
    # * 语句的语法分析 (Syntax / Grammer analysis)
#     nlp_reply += '\n'
#     for i in range(len(responses['tokens'])): 
#         nlp_reply += '\n' + str(responses['tokens'][i])
    
    return nlp_reply

In [42]:
print(KudosData_nlp_generate_reply(responses))

[ NLP 自然语言处理结果 ]

[ 整篇消息 语种 : zh ]
[ 褒贬度 : 0.5 | 语彩累积: 1.1 ]

[ 语句 1: 作为酷豆数据科学的顾问和培训师，Sam Gu （白黑） 结合社群和教育资源，帮助各大公司使用先进的数据科学，机器学习和云计算技术来获取数据洞见。 ]
[ 褒贬度 : 0.8 | 语彩累积 : 0.8 ]
[ 语句 2: 激励下一代人工智能爱好者和领导者。 ]
[ 褒贬度 : 0.2 | 语彩累积 : 0.2 ]

[ 实体 1: 酷豆数据科学 ]
[ 类别 : OTHER | 重要性 : 0.195719 ]
https://en.wikipedia.org/wiki/Data_science
[ 实体 2: 顾问 ]
[ 类别 : PERSON | 重要性 : 0.12852265 ]
[ 实体 3: Sam Gu ]
[ 类别 : PERSON | 重要性 : 0.12153177 ]
[ 实体 4: 培训师 ]
[ 类别 : PERSON | 重要性 : 0.105539404 ]
[ 实体 5: 白黑 ]
[ 类别 : OTHER | 重要性 : 0.105539404 ]
[ 实体 6: 云计算技术 ]
[ 类别 : OTHER | 重要性 : 0.07785424 ]
[ 实体 7: 数据科学 ]
[ 类别 : OTHER | 重要性 : 0.06426745 ]
[ 实体 8: 机器 ]
[ 类别 : CONSUMER_GOOD | 重要性 : 0.052090105 ]
[ 实体 9: 各大公司 ]
[ 类别 : ORGANIZATION | 重要性 : 0.038682967 ]
[ 实体 10: 社群 ]
[ 类别 : OTHER | 重要性 : 0.031572923 ]
[ 实体 11: 教育资源 ]
[ 类别 : OTHER | 重要性 : 0.031572923 ]
[ 实体 12: 人工智能爱好者 ]
[ 类别 : PERSON | 重要性 : 0.023553587 ]
[ 实体 13: 领导者 ]
[ 类别 : PERSON | 重要性 : 0.023553587 ]


### 用微信App扫QR码图片来自动登录

In [48]:
itchat.auto_login(hotReload=True) # hotReload=True: 退出程序后暂存登陆状态。即使程序关闭，一定时间内重新开启也可以不用重新扫码。

In [44]:
# Obtain my own Nick Name
MySelf = itchat.search_friends()
NickName4RegEx = '@' + MySelf['NickName'] + '\s*'

In [45]:
# 自动j进行自然语言分析，以文本形式返回处理结果：
# @itchat.msg_register(TEXT, isGroupChat=True)
@itchat.msg_register(TEXT)
def text_reply(msg):
#     if msg['isAt']:
        text4nlp = msg['Content']
        # call NLP API:
        nlp_responses = KudosData_nlp(text4nlp
                            , parm_nlp_extractDocumentSentiment
                            , parm_nlp_extractEntities
                            , parm_nlp_extractEntitySentiment
                            , parm_nlp_extractSyntax)
        # Format NLP results:
        nlp_reply = KudosData_nlp_generate_reply(nlp_responses)
        print(nlp_reply)
        return nlp_reply

In [46]:
# 在群里，如果收到 @ 自己的文字信息，会自动j进行自然语言分析，以文本形式返回处理结果：
@itchat.msg_register(TEXT, isGroupChat=True)
def text_reply(msg):
    if msg['isAt']:
        text4nlp = re.sub(NickName4RegEx, '', msg['Content'])
        # call NLP API:
        nlp_responses = KudosData_nlp(text4nlp
                            , parm_nlp_extractDocumentSentiment
                            , parm_nlp_extractEntities
                            , parm_nlp_extractEntitySentiment
                            , parm_nlp_extractSyntax)
        # Format NLP results:
        nlp_reply = KudosData_nlp_generate_reply(nlp_responses)
        print(nlp_reply)
        return nlp_reply

In [49]:
itchat.run()

Start auto replying.



Compeleted: NLP analysis API
[ NLP 自然语言处理结果 ]

[ 整篇消息 语种 : zh ]
[ 褒贬度 : 0.4 | 语彩累积: 1.4 ]

[ 语句 1: 在 23 日的首轮比赛中，AlphaGo 以四分之一子的优势，击败了世界第一人类围棋手柯洁。 ]
[ 褒贬度 : 0.5 | 语彩累积 : 0.5 ]
[ 语句 2: 赢得比赛后，这场人机大战引起了人们广泛的关注和讨论。 ]
[ 褒贬度 : 0.4 | 语彩累积 : 0.4 ]
[ 语句 3: DeepMind 也在赛后分析解读了 AlphaGo 背后的技术，表示当前版本 AlphaGo Master 的棋力，较与李世乭对弈的 AlphaGo 版本有三子提升，就连柯洁本人也在微博上表达了自己的「震惊」 ]
[ 褒贬度 : 0.4 | 语彩累积 : 0.4 ]

[ 实体 1: AlphaGo ]
[ 类别 : UNKNOWN | 重要性 : 0.24711089 ]
https://en.wikipedia.org/wiki/AlphaGo
[ 实体 2: 首轮比赛 ]
[ 类别 : OTHER | 重要性 : 0.112002544 ]
[ 实体 3: 优势 ]
[ 类别 : OTHER | 重要性 : 0.091613464 ]
[ 实体 4: 世界 ]
[ 类别 : LOCATION | 重要性 : 0.091613464 ]
[ 实体 5: 人类围棋手 ]
[ 类别 : PERSON | 重要性 : 0.091613464 ]
[ 实体 6: AlphaGo Master ]
[ 类别 : CONSUMER_GOOD | 重要性 : 0.05466258 ]
[ 实体 7: 柯洁 ]
[ 类别 : PERSON | 重要性 : 0.037837576 ]
https://en.wikipedia.org/wiki/Ke_Jie
[ 实体 8: 版本 ]
[ 类别 : CONSUMER_GOOD | 重要性 : 0.032528047 ]
[ 实体 9: 棋力 ]
[ 类别 : OTHER | 重要性 : 0.024161054 ]
[ 实体 10: 人们 ]
[ 类别 : PERSON | 重要性 : 0.023656383 ]
[ 实体 11: 关注 ]
[ 

Bye~


In [51]:
# interupt kernel, then logout
itchat.logout() # 安全退出

<ItchatReturnValue: {'BaseResponse': {'RawMsg': 'logout successfully.', 'ErrMsg': '请求成功', 'Ret': 0}}>

LOG OUT!


### 第四课：自然语言处理：语义和情感分析
### Lesson 4: Natural Language Processing 2
* 消息文字中名称实体的识别 (Name-Entity detection)
* 消息文字中语句的情感分析 (Sentiment analysis, Sentence level)
* 整篇消息文字的情感分析 (Sentiment analysis, Document level)
* 语句的语法分析 (Syntax / Grammer analysis)

### 下一课是:
### 第五课：视频识别和处理
### Lesson 5: Video Recognition & Processing
* 识别视频消息中的物体名字 (Recognize objects in video)
* 识别视频的场景 (Detect scenery in video)
* 直接搜索视频内容 (Search content in video)

<img src='http://www.kudosdata.com/wp-content/uploads/2016/11/cropped-KudosLogo1.png' width=30% style="float: right;">
<img src='reference/WeChat_SamGu_QR.png' width=10% style="float: left;">

